## Ворошилов Владислав
## ПИ20-3
## Семинар 15

In [1]:
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot as plt
import seaborn as sns
import random

### 0. Загрузите датасет для предсказания арендной платы в зависимости от ряда признаков.

In [2]:
df = pd.read_csv("./House_Rent_Dataset.csv")
df.head(5)

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2,10000,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,2022-05-13,2,20000,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2022-05-16,2,17000,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,2022-07-04,2,10000,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,2022-05-09,2,7500,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner


### 1. Превратите признак `Floor` в два признака: `Floor_number`- номер этажа, на котором находится квартира (подвал, полуподвал и цокольный этаж обозначьте числом `0`) и `Total_floors`- общее количество этажей в доме, в котором сдаётся квартира.

In [3]:
df["Floor_number"] = df["Floor"].str.split(" out of ", expand=True)[0]
df.loc[df["Floor_number"] == "Ground", "Floor_number"] = 0
df.loc[df["Floor_number"] == "Upper Basement", "Floor_number"] = 0
df.loc[df["Floor_number"] == "Lower Basement", "Floor_number"] = 0

In [4]:
df["Floor_number"].value_counts()

1     1161
0      961
2      945
3      512
4      272
5      164
6       93
7       74
10      67
8       66
9       65
12      47
11      43
15      41
14      34
18      26
17      22
16      21
19      16
13      15
20      12
25      12
23       9
24       6
21       6
30       5
34       4
28       4
35       3
27       3
22       3
32       3
26       3
65       3
60       3
45       2
36       2
48       2
40       2
53       2
44       1
41       1
46       1
37       1
33       1
50       1
39       1
43       1
29       1
49       1
47       1
76       1
Name: Floor_number, dtype: int64

In [5]:
df["Total_floors"] = df["Floor"].str.split(" out of ", expand=True)[1]
df["Total_floors"].value_counts()

4     938
3     914
2     868
5     422
1     333
     ... 
39      1
54      1
62      1
59      1
89      1
Name: Total_floors, Length: 66, dtype: int64

### 2. Убедитесь, что признаки `Area Type` и `Point of Contact` имеют только по три значения. Удалите строки с теми значениями, которые встречаются менее трёх раз. Оставшиеся значения замените на `0` и `1`. 

In [6]:
df["Area Type"].value_counts()

Super Area     2446
Carpet Area    2298
Built Area        2
Name: Area Type, dtype: int64

In [7]:
df["Point of Contact"].value_counts()

Contact Owner      3216
Contact Agent      1529
Contact Builder       1
Name: Point of Contact, dtype: int64

In [8]:
df.drop(df[df["Area Type"] == "Built Area"].index, axis=0, inplace=True)
df.drop(df[df["Point of Contact"] == "Contact Builder"].index, axis=0, inplace=True)

In [9]:
df["Area Type"].value_counts()

Super Area     2445
Carpet Area    2298
Name: Area Type, dtype: int64

In [10]:
df["Point of Contact"].value_counts()

Contact Owner    3214
Contact Agent    1529
Name: Point of Contact, dtype: int64

In [11]:
df["Area Type"] = df["Area Type"].map({"Super Area": 0, "Carpet Area": 1})

In [12]:
df["Point of Contact"] = df["Point of Contact"].map({"Contact Owner": 0, "Contact Agent": 1})

In [13]:
df["Area Type"].value_counts()

0    2445
1    2298
Name: Area Type, dtype: int64

In [14]:
df["Point of Contact"].value_counts()

0    3214
1    1529
Name: Point of Contact, dtype: int64

### 3. Определите, сколько значений имеет признак `City`. Сделайте из этого признака dummy переменные,т.е. замените этот признак на такое количество признаков, сколько значений он принимает, где в каждом новом признаке на месте его значения, стоят единицы, а остальные - нули.

In [35]:
df["City"].value_counts()

Mumbai       972
Chennai      891
Bangalore    886
Hyderabad    868
Delhi        605
Kolkata      524
Name: City, dtype: int64

In [36]:
df["City"].isna().sum()

0

In [15]:
df = pd.get_dummies(data=df, columns=["City"])
df.head(3)

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact,Floor_number,Total_floors,City_Bangalore,City_Chennai,City_Delhi,City_Hyderabad,City_Kolkata,City_Mumbai
0,2022-05-18,2,10000,1100,Ground out of 2,0,Bandel,Unfurnished,Bachelors/Family,2,0,0,2,0,0,0,0,1,0
1,2022-05-13,2,20000,800,1 out of 3,0,"Phool Bagan, Kankurgachi",Semi-Furnished,Bachelors/Family,1,0,1,3,0,0,0,0,1,0
2,2022-05-16,2,17000,1000,1 out of 3,0,Salt Lake City Sector 2,Semi-Furnished,Bachelors/Family,1,0,1,3,0,0,0,0,1,0


### 4.Замените признак `Furnishing Status` -"статус мебелировки" на числовой признак следующим образом. Замените статус "без мебели" на значение 0, "частичная мебелировка" на 1 и "с мебелью" на значение 2.

In [16]:
df["Furnishing Status"].value_counts()

Semi-Furnished    2251
Unfurnished       1813
Furnished          679
Name: Furnishing Status, dtype: int64

In [17]:
df["Furnishing Status"] = df["Furnishing Status"].map({"Unfurnished": 0, "Semi-Furnished": 1, "Furnished": 2})

In [18]:
df["Furnishing Status"].value_counts()

1    2251
0    1813
2     679
Name: Furnishing Status, dtype: int64

### 5. Признак `Tenant Preferred` замените на два числовых признака, похожим на dummy переменные образом. В первом из них, поставьте единицы там, где значение `Family` и `Bachelors/Family`, а на месте `Bachelors` - нули. Во втором наоборот, единицы там, где `Bachelors` и `Bachelors/Family`, а на месте `Family` - нули.

In [19]:
df["Tenant Preferred"].value_counts()

Bachelors/Family    3441
Bachelors            830
Family               472
Name: Tenant Preferred, dtype: int64

In [21]:
df["Tenant Preferred Family"] = 0
df["Tenant Preferred Bachelors"] = 0

df.loc[(df["Tenant Preferred"] == "Family") | (df["Tenant Preferred"] == "Bachelors/Family"), "Tenant Preferred Family"] = 1
df.loc[(df["Tenant Preferred"] == "Bachelors") | (df["Tenant Preferred"] == "Bachelors/Family"), "Tenant Preferred Bachelors"] = 1


In [22]:
df["Tenant Preferred Family"].value_counts()

1    3913
0     830
Name: Tenant Preferred Family, dtype: int64

In [23]:
df["Tenant Preferred Bachelors"].value_counts()

1    4271
0     472
Name: Tenant Preferred Bachelors, dtype: int64

In [34]:
df.drop(columns=["Tenant Preferred"], inplace=True)

### 6. В дате `Posted On` оставьте только номер месяца, в котором размещено объявление о сдаче в аренду.

In [29]:
df.loc[:, "Posted On"] = df["Posted On"].str[5:7]

In [30]:
df.head(5)

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,Furnishing Status,Tenant Preferred,Bathroom,...,Floor_number,Total_floors,City_Bangalore,City_Chennai,City_Delhi,City_Hyderabad,City_Kolkata,City_Mumbai,Tenant Preferred Family,Tenant Preferred Bachelors
0,05,2,10000,1100,Ground out of 2,0,Bandel,0,Bachelors/Family,2,...,0,2,0,0,0,0,1,0,1,1
1,05,2,20000,800,1 out of 3,0,"Phool Bagan, Kankurgachi",1,Bachelors/Family,1,...,1,3,0,0,0,0,1,0,1,1
2,05,2,17000,1000,1 out of 3,0,Salt Lake City Sector 2,1,Bachelors/Family,1,...,1,3,0,0,0,0,1,0,1,1
3,07,2,10000,800,1 out of 2,0,Dumdum Park,0,Bachelors/Family,1,...,1,2,0,0,0,0,1,0,1,1
4,05,2,7500,850,1 out of 2,1,South Dum Dum,0,Bachelors,1,...,1,2,0,0,0,0,1,0,0,1


In [43]:
df["Posted On"] = df["Posted On"].astype(int)

In [44]:
df.dtypes

Posted On                      int64
BHK                            int64
Rent                           int64
Size                           int64
Area Type                      int64
Furnishing Status              int64
Bathroom                       int64
Point of Contact               int64
Floor_number                  object
Total_floors                  object
City_Bangalore                 uint8
City_Chennai                   uint8
City_Delhi                     uint8
City_Hyderabad                 uint8
City_Kolkata                   uint8
City_Mumbai                    uint8
Tenant Preferred Family        int64
Tenant Preferred Bachelors     int64
dtype: object

### 7. Удалите признак `Area Locality`.

In [32]:
df.drop(columns=["Area Locality"], inplace=True)

### 8. Сохраните полученный датасет и выведете его.

In [36]:
df.drop(columns=["Floor"], inplace=True)

In [37]:
df

,Posted On,BHK,Rent,Size,Area Type,Furnishing Status,Bathroom,Point of Contact,Floor_number,Total_floors,City_Bangalore,City_Chennai,City_Delhi,City_Hyderabad,City_Kolkata,City_Mumbai,Tenant Preferred Family,Tenant Preferred Bachelors
0,05,2,10000,1100,0,0,2,0,0,2,0,0,0,0,1,0,1,1
1,05,2,20000,800,0,1,1,0,1,3,0,0,0,0,1,0,1,1
2,05,2,17000,1000,0,1,1,0,1,3,0,0,0,0,1,0,1,1
3,07,2,10000,800,0,0,1,0,1,2,0,0,0,0,1,0,1,1
4,05,2,7500,850,1,0,1,0,1,2,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4741,05,2,15000,1000,1,1,2,0,3,5,0,0,0,1,0,0,1,1
4742,05,3,29000,2000,0,1,3,0,1,4,0,0,0,1,0,0,1,1
4743,07,3,35000,1750,1,1,3,1,3,5,0,0,0,1,0,0,1,1
4744,07,3,45000,1500,1,1,2,1,23,34,0,0,0,1,0,0,1,0
